## 학습목표
- 감성분석 모델링을 할 수 있다.
- 토큰화/수치화 방법을 이해 할 수 있다.
- konlpy 사용법을 이해 할 수 있다.

#### 감성분석
- 사람의 감정/기분/태도 등을 분석하는 기법
   - case 1 : 감성사전을 이용한 분석(전통적으로 사용하던 방식)
   - case 2 : 인공지능 기술을 이용한 분석(최근 방식)

#### 데이터로딩

In [3]:
import pandas as pd
# 훈련용, 평가용 데이터 로딩
train = pd.read_csv("./data/unsmile_train_v1.0.tsv", # 파일경로
                    delimiter='\t') # 구분자
test = pd.read_csv("./data/unsmile_valid_v1.0.tsv", # 파일경로
                    delimiter='\t') # 구분자

#### 데이터 전처리
- 형태소 단위로 분리
- clean 작업 : 불용어, 반복적인 단어 처리 등 -> 정규표현식 

In [ ]:
# 한국어 형태소 분리 시 자주 활용하는 konlpy를 사용
# 1. jdk, Jpye 등을 설치 -> PC상태에 따라서 안되는 경우 많음 -> colab으로 진행
# 2. konlpy 형태소 분석기 중에 mecab이 있음 -> 리눅스 운영체제에서 사용 가능 -> colab으로 진행

In [18]:
# 문장데이터만 분리하여 파일로 저장
train['문장'].to_csv("./train_origin_text.csv", index=False)
test['문장'].to_csv("./test_origin_text.csv", index=False)

In [20]:
# 'ex05_2 텍스트마이닝 응용_konlpy_사용하기' 파일에서 처리한 데이터 로딩
import pickle

In [24]:
with open("./data/clean_morphs_train.pkl", 'rb') as f :
    clean_morphs_train = pickle.load(f)

with open("./data/clean_morphs_test.pkl", 'rb') as f :
    clean_morphs_test = pickle.load(f)

In [26]:
len(clean_morphs_test)

3737

In [30]:
len(clean_morphs_train)

15005

In [32]:
clean_morphs_train[:5]

[['일안', '시간', '어서', '그런', '아닐까'],
 ['아동',
  '성범죄',
  '페도버는',
  '기록',
  '영원히',
  '고통',
  '는다',
  '무슬림',
  '근친',
  '까지',
  '떨어지',
  '출산',
  '위험'],
 ['루나', '솔로', '앨범', '나왔', '부터', '기운', '진짜'],
 ['어버이',
  '연합',
  '인가',
  '보내',
  '이런',
  '뎃글',
  '는데',
  '이거',
  '어버이',
  '연합',
  '신고',
  '그쪽',
  '에서',
  '고소',
  '가능'],
 ['여기',
  '여자',
  '김치',
  '라고',
  '먼저',
  '불렸',
  '여자',
  '심하',
  '그런다',
  '이렇게',
  '싸우',
  '나쁜',
  '이상',
  '이하',
  '아닌데']]

#### 토큰화 및 수치화(특성추출)
- 토큰화 : 일정 단위로 텍스트를 분리하는 작업
   - 글자(char) : apple -> a / p / p / l / e
   - 단어(word) : 보통 띄어쓰기 기준으로 분리 -> 보통 / 띄어쓰기 / 기준으로 / 분리
   - 형태소 : 동사, 명사, 형용사 등 형태소 기준으로 분리
   - n-gram(유니,바이,트라이 등) : 1/2/3개 씩 단어를 묶어서 토큰화하는 방법
        - 오늘 점신은 맛있는 카레 -> 유니그램 -> 오늘 / 점심은 / 맛있는 / 카레
        - 오늘 점신은 맛있는 카레 -> 바이그램 -> 오늘 점심은 / 점심은 맛있는 / 맛있는 카레
        - 오늘 점신은 맛있는 카레 -> 트라이그램 -> 오늘 점신은 맛있는 / 점심은 맛있는 카레
- 수치화(특성추출) : 의미있는 정보를 담고 있는 숫자형태로 변환하는 방법, 데이터를 정형화 하는 효과가 있다.
   - 빈도기반의 레이블인코딩
   - 원핫인코딩과 유사한 BOW, Tf-idf
   - Word embedding : 딥러닝 학습을 이용해서 수치화 하는 기법

##### BOW(Bag Of word)
- 문장에서 등장하는 단어의 빈도를 측정해 수치화하는 방법
- 단어사전 구축 -> 단어사전을 기반으로 문장내의 단어빈도를 측정
- 장점 : 단순한 알고리즘이라 이해하기가 편하다
- 단점 : 말뭉치에 사용되는 단어가 많으면 부피가 비례해서 커진다 / 문장에서 단어의 순서를 고려x
  (문맥을 파악하는 분석에는 부적합)

In [45]:
from sklearn.feature_extraction.text import CountVectorizer

In [48]:
sample_text = clean_morphs_train[:3]
sample_cv = CountVectorizer() # BOW를 해주는 객체 생성
print(sample_text[0])
print(sample_text[1])
print(sample_text[2])

['일안', '시간', '어서', '그런', '아닐까']
['아동', '성범죄', '페도버는', '기록', '영원히', '고통', '는다', '무슬림', '근친', '까지', '떨어지', '출산', '위험']
['루나', '솔로', '앨범', '나왔', '부터', '기운', '진짜']


In [52]:
# step 1 : 토큰화 및 단어사전 구축
# CountVectorizer 에 토큰화기능이 내장되어있어 문장을 하나로 묶어주는 전처리 작업
sample_text2 = [ " ".join(s) for s in sample_text]
sample_cv.fit(sample_text2)

CountVectorizer()

In [54]:
# 구축된 단어사전 확인 -> 25개 단어등장
sample_cv.vocabulary_

{'일안': 21,
 '시간': 14,
 '어서': 18,
 '그런': 1,
 '아닐까': 15,
 '아동': 16,
 '성범죄': 12,
 '페도버는': 24,
 '기록': 3,
 '영원히': 19,
 '고통': 0,
 '는다': 7,
 '무슬림': 10,
 '근친': 2,
 '까지': 5,
 '떨어지': 8,
 '출산': 23,
 '위험': 20,
 '루나': 9,
 '솔로': 13,
 '앨범': 17,
 '나왔': 6,
 '부터': 11,
 '기운': 4,
 '진짜': 22}

In [56]:
# step 2 : 단어사전을 기반으로 문장내의 단어빈도를 측정
result = sample_cv.transform(sample_text2)
result

<3x25 sparse matrix of type '<class 'numpy.int64'>'
	with 25 stored elements in Compressed Sparse Row format>

In [58]:
# 만약 데이터를 직접 보고 싶다면 활용
result.toarray()

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1,
        0, 0, 0],
       [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0,
        0, 1, 1],
       [0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        1, 0, 0]], dtype=int64)

In [63]:
# 단어사전을 데이터 프레임으로 변환
result_df = pd.DataFrame([sample_cv.vocabulary_.keys()],
                         columns = sample_cv.vocabulary_.values())
result_df

,21,14,18,1,15,16,12,24,3,19,...,8,23,20,9,13,17,6,11,4,22
0,일안,시간,어서,그런,아닐까,아동,성범죄,페도버는,기록,영원히,...,떨어지,출산,위험,루나,솔로,앨범,나왔,부터,기운,진짜


In [70]:
result_df = result_df.sort_index(axis=1)
result_df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,고통,그런,근친,기록,기운,까지,나왔,는다,떨어지,루나,...,아닐까,아동,앨범,어서,영원히,위험,일안,진짜,출산,페도버는


In [72]:
pd.concat([result_df, pd.DataFrame(result.toarray())])

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,고통,그런,근친,기록,기운,까지,나왔,는다,떨어지,루나,...,아닐까,아동,앨범,어서,영원히,위험,일안,진짜,출산,페도버는
0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0
1,1,0,1,1,0,1,0,1,1,0,...,0,1,0,0,1,1,0,0,1,1
2,0,0,0,0,1,0,1,0,0,1,...,0,0,1,0,0,0,0,1,0,0
